In [7]:
import os
import time
import nbimporter

In [8]:
import utils
import blockchain as blockchain_utils
import encryption

### LOAD RESOURCES

In [3]:
settings = utils.load_yaml('resources/settings.yaml')

In [4]:
identifier = utils.load_yaml('resources/identifier.yaml')

In [5]:
latest = utils.load_json('resources/latest.json')

### CONNECT TO THE ETHEREUM GATEWAY

In [ ]:
web3 = blockchain_utils.connect(settings)

### SERIALIZE THE MANAGER CONTRACT

In [7]:
device_manager = blockchain_utils.contract(latest['devicemanager'], web3, settings)

### HASH THE DEVICE IDENTIFIER

In [8]:
hashed_identifier = utils.hash_data(identifier)

### USE THE HASHED IDENTIFIER TO LOCATE THE DEVICES SMART CONTRACT

In [9]:
device_address = device_manager.read({
    'func': 'fetch_device',
    'params': hashed_identifier
})

In [10]:
# IF IT DOES NOT EXIST
if device_address == '0x0000000000000000000000000000000000000000':
    print('THE DEVICE IS NOT REGISTERED')
    sys.exit(0)

### SERIALIZE THE DEVICE SPECIFIC CONTRACT

In [11]:
device_contract = blockchain_utils.contract({
    'address': device_address,
    'abi': latest['device']['abi']
}, web3, settings)

### FETCH & SERIALIZE RSA ENCRYPTION KEY - TEMP SOLUTION

In [9]:
raw_rsa_pubkey, raw_rsa_privkey = encryption.generate_rsa_keys()

In [10]:
# SANITIZE TO SIMULATE BLOCKCHAIN FORMAT
sanitized = encryption.sanitize_key(raw_rsa_pubkey)

In [11]:
# CONVERT TO RSA FORMAT
rsa_pubkey = encryption.serialize_key(sanitized)

### CALLBACK FUNC FOR UPLOADING DATA

In [12]:
def track(file_path):
    
    # OPEN THE FILE
    file = open(file_path, 'r')

    # FIND THE TAILEND OF THE FILE
    file_stats = os.stat(file_path)
    file_size = file_stats[6]
    
    # GOTO THE LAST ROW
    file.seek(file_size)
    
    # DATA CONTAINERS
    hashes = []
    batch = []
        
    # EVENT LOOP
    while True:

        # READ THE LAST LINE
        where = file.tell()
        line = file.readline()

        # IF THE LINE ISNT EMPTY
        if line:

            # EXTRACT THE EVENT PARAMS
            module, code, message, timestamp = utils.parse_line(line)
            
            # HASH THE MODULE & MESSAGE
            hashed = utils.hash_dict({
                'module': module,
                'message': message
            })
            
            # ENCODE THE HASH TO BASE64
            encoded = utils.encode(hashed)
            
            # PUSH HASH & TIMESTAMP
            hashes.append([encoded, timestamp])

            # PUSH FULL ENTRY TO BATCH CONTAINER
            batch.append({
                'hash': encoded,
                'timestamp': timestamp,
                'module': module,
                'code': code,
                'message': message,
            })

        # OTHERWISE, SLEEP FOR A SECOND TO PREVENT SPAM
        else:
            time.sleep(3)

        # IF THE TAILEND OF THE FILE WAS REACHED
        # AND CONTAINER IS NOT EMPTY
        if file.tell() == os.path.getsize(file_path) and len(hashes) > 0 and len(batch) > 0:
            
            # AES ENCRYPT THE BATCH
            sym_data, sym_params = encryption.aes_encrypt({
                'events': batch
            })

            # RSA ENCRYPT AES PARAMS
            asym_params = encryption.rsa_encrypt(sym_params, rsa_pubkey)

            # SUBMIT BATCH DATA TO THE SMART CONTRACT
            device_contract.write({
                'func': 'add',
                'params': [sym_data, asym_params, hashes]
            })

            # PRINT MSG
            print('UPLOADED ', len(batch), 'LINES')

            # RESET CONTAINERS
            hashes = []
            batch = []

### START TRACKING LOG FILE

In [ ]:
track(settings['log_path'])